In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import LdaModel

In [2]:
import cx_Oracle
conn = cx_Oracle.connect("admin", "Onlin3#onlin3#", "adbnews_high")

query = """
    SELECT *
    FROM tb_news_valor_economico
"""
#curr = conn.cursor()
#curr.execute(query)

news_data = pd.read_sql(query, con=conn)
conn.close()

C:\Users\danie\AppData\Local\Temp\ipykernel_16732\2065843896.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  news_data = pd.read_sql(query, con=conn)


In [3]:
import hashlib

news_data["publication_id"] = news_data["URL"].apply(lambda x: hashlib.md5(x.encode()).hexdigest())

In [4]:
# Remova as linhas onde o valor da coluna "SUMMARY" é NaN ou vazio
news_data.dropna(subset=['SUMMARY'], inplace=True)
news_data = news_data[news_data['SUMMARY'].apply(lambda x: x.strip() != '')]

In [5]:
# Defina sua lista de stop words
stop_words = set(stopwords.words('portuguese'))

In [6]:
# Defina uma função para remover stop words
def remove_stop_words(text):
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

In [7]:
# Transformando em minusculas
news_data['NEW_SUMMARY'] = news_data['SUMMARY'].apply(str.lower)

In [8]:
# Defina a lista de pontuação para remoção
import string
punctuation = string.punctuation

In [9]:
# Defina uma função para remover a pontuação
def remove_punctuation(text):
    return ''.join([char for char in text if char not in punctuation])

In [10]:
# Aplique a função remove_punctuation à coluna NEW_SUMMARY do dataframe
news_data['NEW_SUMMARY'] = news_data['NEW_SUMMARY'].apply(remove_punctuation)

In [11]:
# Aplique a função de remoção de stop words à coluna de texto da base de dados
news_data['text_processed'] = news_data['NEW_SUMMARY'].apply(remove_stop_words)

In [12]:
# Defina o objeto de processamento de stemming
stemmer = PorterStemmer()

In [13]:
# Defina uma função para pré-processar os dados
def preprocess(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

In [14]:
# Crie um objeto CountVectorizer
vectorizer = CountVectorizer()

In [15]:
# Aplique a função de pré-processamento à coluna de texto da base de dados
news_data['text_stemmed'] = news_data['text_processed'].apply(preprocess)

In [16]:
# Aplique o CountVectorizer à coluna de texto processado da base de dados
dtm = vectorizer.fit_transform(news_data['text_stemmed'])

In [17]:
# Converta o Document-Term Matrix para um objeto gensim corpus
corpus = gensim.matutils.Sparse2Corpus(dtm)

In [18]:
# Crie um objeto Dictionário
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

In [19]:
# Crie o modelo LDA
lda = LdaModel(corpus=corpus, id2word=id2word, num_topics=10, passes=50)

In [20]:
# Visualize os tópicos identificados pelo LDA
for topic in lda.print_topics():
    print(topic)

(0, '0.014*"empregará" + 0.008*"excepcionalidad" + 0.008*"administrativa" + 0.008*"172080" + 0.008*"celebrava" + 0.008*"entregu" + 0.008*"azar" + 0.008*"coreana" + 0.008*"gued" + 0.007*"depredaçõ"')
(1, '0.009*"carbocloro" + 0.009*"13715" + 0.008*"fratura" + 0.005*"eventualment" + 0.005*"7317" + 0.005*"convocaria" + 0.005*"escalado" + 0.005*"lament" + 0.005*"demissõ" + 0.005*"caiu"')
(2, '0.007*"atrapalh" + 0.004*"criará" + 0.004*"001" + 0.004*"espontânea" + 0.004*"am" + 0.004*"1431" + 0.004*"brasília" + 0.004*"infantil" + 0.004*"acertando" + 0.004*"acompanharam"')
(3, '0.006*"frota" + 0.005*"cratera" + 0.005*"2009" + 0.005*"irritado" + 0.005*"18231" + 0.005*"festiv" + 0.005*"151" + 0.005*"honra" + 0.005*"835" + 0.005*"candidato"')
(4, '0.011*"imprudent" + 0.008*"decidirem" + 0.008*"mango" + 0.008*"importado" + 0.008*"causar" + 0.008*"agrad" + 0.008*"cobra" + 0.008*"deixaram" + 0.008*"dstribuidora" + 0.008*"devida"')
(5, '0.012*"encerrando" + 0.009*"exager" + 0.009*"100" + 0.005*"altis

NEXT
https://www.youtube.com/watch?v=TKjjlp5_r7o